# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    # Let's find lines defining present lane. For that, we extend all line to the bottom of the screen
    # Let's convert all lines to their slope/intercept configuration (y = mx + b)
    ms = []
    bs = []
    max_y = img.shape[0]
    bottom_intrp = []
    try:        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            m = (y2 - y1) / (x2 - x1)
            # only append those lines that are not too vertical or too horizontal
            if (abs(m) < 5) and (abs(m) > 0.2):
                ms.append(m)
                b = -x1 * m + y1
                bs.append(b)
                bottom_intrp.append((max_y - b) / m)
            else:
                continue
            #cv2.line(img, (x1, y1), (x2, y2), [0, 0, 255], thickness)
        slopes = np.array(ms)
        y_intercept = np.array(bs)
        b_intercept = np.array(bottom_intrp)
        # Calculation of center of image to select most likely lines
        # This seems ugly. It should be rewritten
        dist_to_center = b_intercept - (img.shape[1] / 2)
        # Selecting each side of lines
        left_slopes = slopes[dist_to_center < 0]
        rigth_slopes = slopes[dist_to_center > 0]
        left_bs = y_intercept[dist_to_center < 0]
        right_bs = y_intercept[dist_to_center > 0]
        left_dist = dist_to_center[dist_to_center < 0]
        right_dist = dist_to_center[dist_to_center > 0]
        left_bottom = b_intercept[dist_to_center < 0]
        right_bottom = b_intercept[dist_to_center > 0]
        # Selecting only a few lines closest to the center
        sel_lines = 2
        selected_left = left_dist.argsort()[-sel_lines:]
        selected_right = right_dist.argsort()[0:sel_lines]
        # Uncomment next 2 lines to take all detected and filtered lines into account
        #selected_left = left_dist.argsort()
        #selected_right = right_dist.argsort()
        # Combining the selected lines
        left_slope = left_slopes[selected_left].mean()
        rigth_slope = rigth_slopes[selected_right].mean()
        left_intercept = left_bs[selected_left].mean()
        right_intercept = right_bs[selected_right].mean()
        x_left_bottom = left_bottom[selected_left].mean()
        x_right_bottom = right_bottom[selected_right].mean()
        # Find intersection point to use as final point in the line
        x = -(left_intercept - right_intercept) / (left_slope - rigth_slope)
        y = x * left_slope + left_intercept
        global average
        if sum(average) == 0:
            x1_left = int(x_left_bottom)
            x2_right = int(x_right_bottom)
            x3 = int(x)
            y3 = int(y)
        else:
            alpha = 0.05
            x1_left = int(x_left_bottom * alpha + average[0] * (1 - alpha))
            x2_right = int(x_right_bottom * alpha + average[1] * (1 - alpha))
            x3 = int(x * alpha + average[2] * (1 - alpha))
            y3 = int(y * alpha + average[3] * (1 - alpha))
    except:
        # Some error in line detection
        x1_left, x2_right, x3, y3 = average
    average = [x1_left, x2_right, x3, y3]
    cv2.line(img, (x1_left, max_y), (x3, y3), color, thickness)
    cv2.line(img, (x2_right, max_y), (x3, y3), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """ `img` should be the output of a Canny transform.
    Returns an image with hough lines drawn.  """
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

images_list = os.listdir('test_images/')

def proccess_im(image):
    low_thres = 180
    high_thres = 255

    white_low = np.array([low_thres, low_thres, low_thres])
    white_high = np.array([high_thres, high_thres, high_thres])
    yellow_low = np.array([low_thres, low_thres, 0])
    yellow_high = np.array([high_thres, high_thres, high_thres])

    # Threshold the image to select white lanes
    white = cv2.inRange(image, white_low, white_high)
    # Threshold the image to select yellow lanes
    yellow = cv2.inRange(image, yellow_low, yellow_high)
    # Combine both masks
    im = white + yellow
    imshape = image.shape
    ignore_mask_color = 0
    vertices = np.array([[(0.45 * imshape[1], 0.55 * imshape[0]),
                          (0.1 * imshape[1], 1.0 * imshape[0]), 
                          (0.9 * imshape[1], 1.0 * imshape[0]), 
                          (0.55 * imshape[1], 0.55 * imshape[0])]], dtype=np.int32)
    masked_im = region_of_interest(im, vertices)
    kernel_size = 3 # Must be an odd number (3, 5, 7...)
    blur_im = cv2.GaussianBlur(masked_im,(kernel_size, kernel_size),0)
    edges = canny(blur_im, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, 1/180 * np.pi, 10, np.array([]), 10, 50)
    #draw line on top of image
    draw_lines(image, lines)
    return image

for test_im in images_list:
    average = [0, 0, 0, 0]
    org_image = mpimg.imread('test_images/' + test_im)
    mpimg.imsave('test_images/out_' + test_im, proccess_im(org_image))

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    low_thres = 200
    high_thres = 255

    white_low = np.array([low_thres, low_thres, low_thres])
    white_high = np.array([high_thres, high_thres, high_thres])
    yellow_low = np.array([low_thres, low_thres, 0])
    yellow_high = np.array([high_thres, high_thres, high_thres])

    # Threshold the image to select white lanes
    white = cv2.inRange(image, white_low, white_high)
    # Threshold the image to select yellow lanes
    yellow = cv2.inRange(image, yellow_low, yellow_high)
    # Combine both masks
    im = white + yellow
    edges = canny(im, 50, 150)
    imshape = image.shape
    ignore_mask_color = 0  
    vertices = np.array([[(0.45 * imshape[1], 0.6 * imshape[0]),
                          (0.1 * imshape[1], 1.0 * imshape[0]), 
                          (0.9 * imshape[1], 1.0 * imshape[0]), 
                          (0.55 * imshape[1], 0.6 * imshape[0])]], dtype=np.int32)
    masked_im = region_of_interest(im, vertices)
    kernel_size = 3 # Must be an odd number (3, 5, 7...)
    blur_im = cv2.GaussianBlur(masked_im,(kernel_size, kernel_size),0)
    edges = canny(blur_im, 50, 150)
    # submited values:
    # lines = cv2.HoughLinesP(edges, 1, 1/180 * np.pi, 10, np.array([]), 10, 150)
    # recomended values bellow
    lines = cv2.HoughLinesP(edges, 1, 1/180 * np.pi, 30, np.array([]), 100, 160)

    #draw line on top of image
    draw_lines(image, lines)
    result = image
    
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
average = [0, 0, 0, 0]
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
average = [0, 0, 0, 0]
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


One of the first ways I would try to make the algorithm more robust would be to make two passes. By looking at the video before we select which lines to use it seems to be detecting the lane better than it is captured by just two line. It seems an interesting root to create an image with thick line in a clearly identifiable color and do a new pass. This time, the distance between lines should be bigger as we would be trying to bridge the gap between them. Hopefully, by combining thick line with clearly identifiable color (bright orange for example) we would smooth out the lane without introducing noise or artifacts in its detection. It may be interesting to see if this can capture curved lanes without having to draw over them. This cost computation time so it may not be really worth it.
As it stands right now, the algorithm filters lines close to vertical. This was introduced for robustness. However, that would keep it from detecting lanes during a lane change. This should be treated better.
Opencv api is so extend that I'm positive this could be written in a more succinct way.  It is also it extension that makes it somehow hard to find that function that helps. I also found some of the documentation related to python lacking. There seems to be more documentation oriented towards c++. It makes sense since we are talking about a wrapper.
Overall I enjoy the exercise. I’m not too happy with may code because it seem unorganized. I started experimenting right away and in the end I never got the time to organize small functions that would have help quite a bit. At that point I was focused on detecting the lanes as well as I could instead of organizing the code. 

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
average = [0, 0, 0, 0]
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))